# Convolutional Neural Network - Getting Started

In this notebook we will look at how we can load one of the pre-trained models we talked about in class and run soem predictions on one of our own images

In [ ]:
from torchvision import datasets
from torchvision import models
from torchvision import transforms
import torch
import json
import re
from pprint import pprint
from torchsummary import summary

## Lets load a Resnet101 pretrained model

In [ ]:
model = models.resnet101(pretrained=True)

### PyTorch Transforms

Pytorch Vision has multiple transforms that we can use together. We call `Compose` to chain them together

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

### Loading an image
Lets load the same image of my puppy from the lecture

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
img = Image.open('../../data/dog.png')

In [ ]:
print(model)

In [ ]:
img = img.convert('RGB')

In [ ]:
plt.imshow(img)

Here we call pre process and the image will run through all the transforms

In [ ]:
imgt = preprocess(img)

In [ ]:
imgt.shape

In [ ]:
batch = torch.unsqueeze(imgt, 0)

In [ ]:
batch.shape

Lets put the model in evaluation mode. Which eliminates the overhead of gradient optimizations

In [ ]:
model.eval()

Next we perform inference to get the prediction.

In [ ]:
preds = model(batch)

### Model architecture

What does ResNet 101 look like

In [ ]:
summary(model, input_size=(3, 224, 224))

In [ ]:
preds.shape

In [ ]:
preds

### ImageNet Classes
Next we load the 1000 image classes of ImageNet to see what we predicted

In [ ]:
classes = dict()
with open('../../data/imagenet_labels.txt', 'r') as f:
    for line in f.readlines():
        parts = line.split(':')
        classes[int(parts[0])] = re.sub("'|\n", '', parts[1])

Next we get the maximum predicted value

In [ ]:
_, pred = torch.max(preds, 1)

In [ ]:
pred

### Lets use Softmax to generate the classifications

In [ ]:
confidence = torch.nn.functional.softmax(preds, dim=1)[0] * 100

In [ ]:
classes[int(pred[0])]

In [ ]:
confidence[pred[0]].item()

What about reviewing the top 5 probabilities

In [ ]:
_, indices = torch.sort(preds, descending=True)
[(classes[int(idx)], confidence[int(idx)].item()) for idx in indices[0][:5]]

### Lets look at visualizing our feature maps

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

model.conv1.register_forward_hook(get_activation('conv1'))
output = model(batch)

act = activation['conv1'].squeeze()
fig, axarr = plt.subplots(act.size(0), figsize=(200,200))
for idx in range(act.size(0)):
    axarr[idx].imshow(act[idx])


# Excercises

1. Try a different model
2. Try a different dataset
3. Feed in your own model
4. See if you can fool the neural network with images outside of the dataset labels